In [1]:
from __future__ import print_function, division, absolute_import

import argparse
import time

import numpy as np
seed = 7
np.random.seed(seed)

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
os.chdir('../CoBind')

import evolutron.networks as nets
from evolutron.motifs import motif_extraction
from evolutron.tools import load_dataset, none2str, Handle
from evolutron.engine import DeepTrainer

Using Theano backend.


In [2]:
# Data and model
data_id = 'type2p'
padded = True
mode = 'train'

# Architecture
n_filters = 64
filter_length = 10
n_convs = 1
n_fc = 1

# Training
num_epochs = 1
batch_size = 50
optimizer = 'nadam'
rate = .01
validate = .2

In [3]:
dataset = load_dataset(data_id, padded=padded, codes=True)
x_data, y_data = dataset
if type(x_data) == np.ndarray:
    input_shape = x_data[0].shape
elif type(x_data) == list:
    input_shape = (None, x_data[0].shape[1])
else:
    raise TypeError('Something went wrong with the dataset type')

Dataset size: 3595


In [4]:
print('Building model ...')
net_arch = nets.DeepDNABind.from_options(input_shape, n_conv_layers=n_convs, n_fc_layers=n_fc,
                                       n_filters=n_filters, filter_length=filter_length)
conv_net = DeepTrainer(net_arch)
conv_net.compile(optimizer=optimizer, lr=rate)
conv_net.display_network_info()

Building model ...
Neural Network has 18584 trainable parameters
  #  Name            Shape      Parameters
---  --------------  -------  ------------
  0  aa_seq          657x20              0
  1  Conv1           657x64          12864
  2  maxpooling1d_1  1x64                0
  3  flatten_1       64                  0
  4  FCEnc1          64               4160
  5  Classifier1     4                 260
  6  Classifier2     4                 260
  7  Classifier3     4                 260
  8  Classifier4     4                 260
  9  Classifier5     4                 260
 10  Classifier6     4                 260


In [5]:
print('Started training at {}'.format(time.asctime()))
conv_net.fit(x_data, np.split(y_data, 6, axis=1),
             nb_epoch=num_epochs,
             batch_size=batch_size,
             validate=validate)

Started training at Tue Nov 22 18:51:27 2016
Train on 2876 samples, validate on 719 samples
Epoch 1/1
2876/2876 [==============================] - 12s - loss: 5.2243 - Classifier1_loss: 0.8113 - Classifier2_loss: 0.9771 - Classifier3_loss: 0.8269 - Classifier4_loss: 0.8292 - Classifier5_loss: 0.9777 - Classifier6_loss: 0.8022 - Classifier1_categorical_accuracy: 0.5918 - Classifier2_categorical_accuracy: 0.5518 - Classifier3_categorical_accuracy: 0.5076 - Classifier4_categorical_accuracy: 0.4993 - Classifier5_categorical_accuracy: 0.5921 - Classifier6_categorical_accuracy: 0.5918 - val_loss: 3.6127 - val_Classifier1_loss: 0.6255 - val_Classifier2_loss: 0.5750 - val_Classifier3_loss: 0.5921 - val_Classifier4_loss: 0.5977 - val_Classifier5_loss: 0.5889 - val_Classifier6_loss: 0.6336 - val_Classifier1_categorical_accuracy: 0.6662 - val_Classifier2_categorical_accuracy: 0.7455 - val_Classifier3_categorical_accuracy: 0.5396 - val_Classifier4_categorical_accuracy: 0.5160 - val_Classifier5_cat

In [7]:
conv_net.history.history

{'Classifier1_categorical_accuracy': [0.59179416126850082],
 'Classifier1_loss': [0.81128331938025355],
 'Classifier2_categorical_accuracy': [0.5518080663500724],
 'Classifier2_loss': [0.97709349262034606],
 'Classifier3_categorical_accuracy': [0.50764951521792401],
 'Classifier3_loss': [0.82690732989341065],
 'Classifier4_categorical_accuracy': [0.4993045927026673],
 'Classifier4_loss': [0.82919067641790123],
 'Classifier5_categorical_accuracy': [0.59214186489126119],
 'Classifier5_loss': [0.977677746716063],
 'Classifier6_categorical_accuracy': [0.59179416070893009],
 'Classifier6_loss': [0.80216827936812474],
 'loss': [5.2243208714420177],
 'lr': [array(0.009999999776482582, dtype=float32)],
 'val_Classifier1_categorical_accuracy': [0.66620306378445471],
 'val_Classifier1_loss': [0.62553389016046645],
 'val_Classifier2_categorical_accuracy': [0.74547982671827862],
 'val_Classifier2_loss': [0.57499778055183082],
 'val_Classifier3_categorical_accuracy': [0.53963838391244323],
 'val_Cl

In [15]:
import re
pattern = re.compile("val_Classifier[0-9]_loss")
np.mean([v[-1] for k,v in conv_net.history.history.items() if pattern.match(k)])

0.6021231660411267